<a href="https://colab.research.google.com/github/EliasNoorzad/Pose6d_project/blob/main/dataset/yolo_conversion_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The script walks through every image ID listed in the train.txt and test.txt files, finds that frame’s bounding-box info in gt.yml, converts the box from pixel values to YOLO’s normalised format ( xc , yc , w , h ) and saves the four numbers in a matching .txt file inside a new labels folder. When it finishes, every image across all objects now has a YOLO-style label file ready for training.

In [ ]:
OBJ_DIR   = "/content/drive/MyDrive/Linemod/Linemod_preprocessed/data/15" # obj_01 folder
TRAIN_TXT = OBJ_DIR + "/train.txt"                                     # training IDs
VAL_TXT   = OBJ_DIR + "/test.txt"                                      # validation IDs

import os, yaml, pathlib

# 1. make the label folder once
label_dir = pathlib.Path(OBJ_DIR, "labels")
label_dir.mkdir(exist_ok=True)

# 2. load gt.yml one time
gt_path = pathlib.Path(OBJ_DIR, "gt.yml")
with open(gt_path, "r") as f:
    gt_all = yaml.safe_load(f)          # dict, keys are '0','1',...

# 3. helper that writes one .txt file
def write_label(id_str):
    pose = gt_all[int(id_str)][0]       # first (only) object
    x, y, w, h = pose["obj_bb"]         # four integers from gt.yml

    img_w, img_h = 640, 480             # line-MOD RGB size
    x_c   = (x + w / 2.0) / img_w
    y_c   = (y + h / 2.0) / img_h
    w_n   = w / img_w
    h_n   = h / img_h

    txt_path = pathlib.Path(label_dir, f"{id_str}.txt")
    with open(txt_path, "w") as f_txt:
        f_txt.write(f"12 {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

# 4. loop over every ID in train.txt and test.txt
for txt_file in (TRAIN_TXT, VAL_TXT):
    with open(txt_file, "r") as f:
        for line in f:
            img_id = line.strip()       # '0004'
            if img_id:                  # skip empty lines
                write_label(img_id)

print("✓  YOLO label files are in:", label_dir)


The code just turns the bare numeric IDs in train.txt and test.txt into full relative file-paths that YOLO can actually load. It reads each line, wraps it with the folder prefix “rgb/” and the file extension “.png” (so 0004 becomes rgb/0004.png), and then saves the resulting lists to train_images.txt and val_images.txt. Those two new text files now hold the exact image paths that the training script will point at later.

In [ ]:
OBJ_DIR   = "/content/drive/MyDrive/Linemod/Linemod_preprocessed/data/15" # obj_01 folder
TRAIN_TXT = OBJ_DIR + "/train.txt"                                     # training IDs
VAL_TXT   = OBJ_DIR + "/test.txt"

# Open the input file (change 'input.txt' to your filename)
with open(TRAIN_TXT, "r") as infile:
    lines = infile.readlines()

# Process each line
modified_lines = [f"rgb/{line.strip()}.png\n" for line in lines]

# Write to a new output file
with open(OBJ_DIR + "/train_images.txt", "w") as outfile:
    outfile.writelines(modified_lines)


In [ ]:
# Open the input file (change 'input.txt' to your filename)
with open(VAL_TXT, "r") as infile:
    lines = infile.readlines()

# Process each line
modified_lines = [f"rgb/{line.strip()}.png\n" for line in lines]

# Write to a new output file
with open(OBJ_DIR + "/val_images.txt", "w") as outfile:
    outfile.writelines(modified_lines)

**Turning data suitable for YOLO**

Using the two text files that list our training and validation frame IDs, we simply copied each RGB image together with its matching YOLO label into the directory layout that Ultralytics expects: train/images & train/labels for learning, and val/images & val/labels for evaluation. After this one-step transfer, every frame mentioned in the lists now lives in the right sub-folder alongside its annotation, so the multi-object dataset is organised and ready for YOLO-v8 training.

In [ ]:
import shutil
from pathlib import Path

BASE = Path("/content/drive/MyDrive/Linemod/Linemod_preprocessed")
SRC = BASE / "data/15"
DST = BASE / "yolo_data/15"

def copy_images_and_labels(list_file, split):
    with open(SRC / list_file, "r") as f:
        filenames = [line.strip().split("/")[-1] for line in f]  # get '0933.png'

    for name in filenames:
        img_src = SRC / "rgb" / name
        img_dst = DST / split / "images" / name
        shutil.copy(img_src, img_dst)

        label_name = name.replace(".png", ".txt")
        label_src = SRC / "labels" / label_name
        label_dst = DST / split / "labels" / label_name
        shutil.copy(label_src, label_dst)

# Copy train
copy_images_and_labels("train_images.txt", "train")

# Copy val
copy_images_and_labels("val_images.txt", "val")


We ran a short script that adds the object’s ID to the start of every image- and label-filename. It walks through each object’s train/images, train/labels, val/images, and val/labels folders and renames files like 0008.png to 15_0008.png (and the matching text file to 15_0008.txt). Nothing inside the files changes—only their names—so each label still matches its picture. After doing this for every object, all files have unique names, so they can sit in one big folder without clashing.

In [ ]:
import os

YOLO_ROOT = "/content/drive/MyDrive/Linemod/Linemod_preprocessed/yolo_data/15"
obj_id = "15"

# loop over train/val and images/labels
for split in ("train", "val"):
    for kind in ("images", "labels"):
        folder = os.path.join(YOLO_ROOT, split, kind)
        if not os.path.isdir(folder):
            continue

        for filename in os.listdir(folder):
            old_path = os.path.join(folder, filename)
            new_name = f"{obj_id}_{filename}"
            new_path = os.path.join(folder, new_name)
            os.rename(old_path, new_path)

print("All files renamed with prefixes!")


This helper script merges each object’s YOLO-formatted folders into one master dataset. After you set OBJ_ID (e.g., “15”), it loops over the object’s train and val splits and their images / labels sub-folders, making matching directories inside a central yolo/train/… and yolo/val/… structure. Every image and its label are then copied—not moved—into these master folders, so the original per-object data stay intact while the combined dataset grows with uniquely prefixed files ready for multi-class training.

In [ ]:
import os
import shutil


OBJ_ID    = "15"    # e.g. "01", "02", … change and rerun for each object
SRC_ROOT  = "/content/drive/MyDrive/Linemod/Linemod_preprocessed/yolo_data/"     # where your per-object folders live
DEST_ROOT = "/content/drive/MyDrive/Linemod/Linemod_preprocessed/yolo"           # your central yolo/ folder

for split in ("train", "val"):
    for kind in ("images", "labels"):
        src_dir = os.path.join(SRC_ROOT, OBJ_ID, split, kind)
        dst_dir = os.path.join(DEST_ROOT, split, kind)
        os.makedirs(dst_dir, exist_ok=True)
        for fn in os.listdir(src_dir):
            shutil.copy(
                os.path.join(src_dir, fn),
                os.path.join(dst_dir, fn)
            )

print(f"✅ Object {OBJ_ID} merged into `{DEST_ROOT}/{split}/`")
